In [1]:
# load data
tips_data_path = 'C:/Users/Aditya/Documents/GitHub/neu/nlp/LDA_Explore/input/yelp_academic_dataset_tip.json'
reviews_data_path = 'C:/Users/Aditya/Documents/GitHub/neu/nlp/LDA_Explore/output/useful_reviews_4.json'
restaurant_data_path = 'C:/Users/Aditya/Documents/GitHub/neu/nlp/LDA_Explore/output/restaurants.json'
stemmed_restaurant_tips_data_path = 'output/stemmed_restaurant_tips.json'
stemmed_restaurant_reviews_data_path = 'output/stemmed_restaurant_reviews.json'

restaurant_tips_data_path = 'output/restaurant_tips.json'
restaurant_reviews_data_path = 'output/restaurant_reviews.json'

pos_tagged_restaurant_reviews_data_path = 'output/pos_tagged_restaurant_reviews.json'
pos_tagged_restaurant_tips_data_path = 'output/pos_tagged_restaurant_tips.json'


In [2]:
# Methods of data processing ,phase 2

import json

import nltk
ps = nltk.PorterStemmer()
def get_stemmed_text(text):
    tokens = nltk.word_tokenize(text)
    stemmed_tokens = [ps.stem(x) for x in tokens]
    return ' '.join(stemmed_tokens)


def get_restaurant_ids(filename):
    restaurant_ids = []
    with open(filename,'r',encoding='utf-8') as f:
        for line in f:
            row = json.loads(line)
            business_id = row['business_id']
            restaurant_ids.append(business_id)
    
    return set(restaurant_ids)

def get_restaurant_stars(filename):
    restaurant_stars = {}
    with open(filename,'r',encoding='utf-8') as f:
        for line in f:
            row = json.loads(line)
            business_id = row['business_id']
            restaurant_stars[business_id] = row['stars']
    return restaurant_stars
def get_restaurants_tips_reviews_data(filename,restaurant_ids,keys,stem=True):
    tip_review_data_list = []
    with open(filename,'r',encoding='utf-8') as f:
        i = 0
        for line in f:
            if i % 10000 == 0:
                print(i)
            i+=1
            row = json.loads(line)
            business_id = row['business_id']
            if business_id in restaurant_ids:
                if stem:
                    row['text'] = get_stemmed_text(row['text'])
                required_row = {key:row[key] for key in keys}
                tip_review_data_list.append(required_row)
    
    return tip_review_data_list

def write_stemmed_text_for_tips(output_file_path,tip_review_data_list):
    with open(output_file_path,'w',encoding='utf-8') as f:
        for tip_review_data in tip_review_data_list:
            f.write(json.dumps(tip_review_data) + '\n')
            
from collections import defaultdict
from nltk import pos_tag

def load_stemmed_user_reviews_by_business(filename):
    business_reviews = defaultdict(list)
    with open(filename,'r',encoding='utf-8') as f:
        for line in f:
            row = json.loads(line)
            business_id = row['business_id']
            business_reviews[business_id].append(row['text'])
    
    return business_reviews

def load_pos_tagged_reviews_by_business(filename):
    pos_tagged_reviews = {}
    with open(filename,'r',encoding='utf-8') as f:
        for line in f:
            row = json.loads(line)
            review_id = row['review_id']
            pos_tagged_reviews[review_id] = row['text']

    return pos_tagged_reviews

def transform_review_text_pos_tags(input_filename, output_filename):
    business_reviews = []
    with open(input_filename, 'r', encoding='utf-8') as f:
        i = 0
        for line in f:
            if i % 100 == 0:
                print(i)

            i += 1
            row = json.loads(line)
            pos_tags = pos_tag(row['text'].split(' '))
            joined_pos_tags = [x[0] + '/' + x[1] for x in pos_tags]
            row['text'] = ' '.join(joined_pos_tags)
            business_reviews.append(row)

    with open(output_filename, 'w', encoding='utf-8') as f:
        for row in business_reviews:
            f.write(json.dumps(row) + '\n')



In [3]:
# restaurant_ids = get_restaurant_ids(restaurant_data_path)
# business_tip_text = get_restaurants_tips_reviews_data(tips_data_path,restaurant_ids,['business_id','text'])
# write_stemmed_text_for_tips(stemmed_restaurant_tips_data_path,business_tip_text)
# business_tip_text = get_restaurants_tips_reviews_data(tips_data_path,restaurant_ids,['business_id','text'],stem=False)
# write_stemmed_text_for_tips(restaurant_tips_data_path,business_tip_text)
print('hello')

hello


In [4]:
#business_review_text = get_restaurants_tips_reviews_data(reviews_data_path,restaurant_ids,['business_id','review_id','text','stars'])
# write_stemmed_text_for_tips(stemmed_restaurant_reviews_data_path,business_review_text)
# business_review_text = get_restaurants_tips_reviews_data(reviews_data_path,restaurant_ids,['business_id','review_id','text','stars'],stem=False)
# write_stemmed_text_for_tips(restaurant_reviews_data_path,business_review_text)

In [18]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def print_top_words(model, feature_names, n_top_words):
    print(len(model.components_))
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print("Done")
    
    
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print("Documents -----")
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print ("Document %d:" % (doc_index))
            print (documents[doc_index])
            print("-------")

def extract_topics_using_lda(list_of_text,vectorizer,n_components,max_iter,no_top_documents):
    doc_vectorizer = vectorizer(stop_words='english')
    t0 = time()
    doc_vector = doc_vectorizer.fit_transform(list_of_text)
    print("done in %0.3fs." % (time() - t0))
    t0 = time()
    lda = LatentDirichletAllocation(n_components=n_components, max_iter=max_iter,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0).fit(doc_vector)
    
    
    print("done in %0.3fs." % (time() - t0))
    print("\nTopics in LDA model:")
    lda_W = lda.transform(doc_vector)
    lda_H = lda.components_
    feature_names = doc_vectorizer.get_feature_names()
    display_topics(lda_H,lda_W,feature_names,list_of_text,50,no_top_documents)
    return doc_vectorizer,lda



def extract_topics_using_nfm(list_of_text,vectorizer,n_components,max_iter):
    doc_vectorizer = vectorizer(stop_words = 'english')
    t0 = time()
    doc_vector = doc_vectorizer.fit_transform(list_of_text)
    print("done in %0.3fs." % (time() - t0))
    t0 = time()
    nmf = NMF(n_components=n_components, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(doc_vector)
    print("done in %0.3fs." % (time() - t0))
    print("\nTopics in NFM model:")
    feature_names = doc_vectorizer.get_feature_names()
    print_top_words(nmf, feature_names, 10)
    return doc_vectorizer,nmf



In [19]:
def sentiment_pos_distribution(list_of_docs):
    """ Look at only adjectives and noun POS-word pairs for the document
        Ref:https://link.springer.com/content/pdf/10.1007/978-3-642-22606-9_33.pdf
        Ref:https://www.researchgate.net/publication/272863616_Sentiment_Analysis_of_Movie_Reviews_using_POS_tags_and_Term_Frequencies
    """
    t0 = time()
    word_pairs = set()
    word_pair_features = [defaultdict(lambda: 0) for x in range(len(list_of_docs))]
    print(word_pair_features)
    for idx, doc in enumerate(list_of_docs):
        for sent in nltk.sent_tokenize(doc):
            if sent.strip().count(' ') <= 3: continue  # make sure sent has 3 words atleast
            tagged_sent = nltk.word_tokenize(sent)
            postags = nltk.pos_tag(tagged_sent)
            for word_tag_pairs in nltk.ngrams(postags, 3):
                wtp1, wtp2, wtp3 = word_tag_pairs
                if ((wtp1[1] == 'JJ' and (wtp2[1] == 'NN' or wtp2[1] == 'NNS')) or
                    ((wtp1[1] == 'RB' or wtp1[1] == 'RBR' or wtp1[1] == 'RBS') and (wtp2[1] == 'JJ') and (wtp3[1] != 'NN' or wtp3[1] != 'NNS')) or
                    ((wtp1[1] == 'JJ') and (wtp2[1] == 'JJ') and (wtp3[1] != 'NN' or wtp3[1] != 'NNS')) or
                    ((wtp1[1] == 'NN' or wtp1[1] == 'NNS') and (wtp2[1] == 'JJ') and (wtp3[1] != 'NN' or wtp3[1] != 'NNS')) or
                    ((wtp1[1] == 'RB' or wtp1[1] == 'RBR' or wtp1[1] == 'RBS') and (wtp2[1] == 'VB' or wtp2[1] == 'VBD' or wtp2[1] == 'VBN' or wtp2[1] == 'VBG'))):

                    pair_key = f"{wtp1[0]}-{wtp2[0]}"
                    word_pairs.add(pair_key)
                    word_pair_features[idx][pair_key] += 1

    print("sent features done in %0.3fs." % (time() - t0))
    return (word_pair_features, list(word_pairs))


def review_pos_distribution_vector(review_docs):
    word_pair_features_for_review = {review_id: defaultdict(lambda:0) for review_id in review_docs}
    t0 = time()
    word_pairs = set()
    for review_id in review_docs:
        pos_tagged_review = review_docs[review_id]
        pos_tagged_review = pos_tagged_review.split(" ")
        pos_tagged_review = [x.split('/') for x in pos_tagged_review]
        for word_tag_pairs in nltk.ngrams(pos_tagged_review, 3):
                wtp1, wtp2, wtp3 = word_tag_pairs
                if ((wtp1[1] == 'JJ' and (wtp2[1] == 'NN' or wtp2[1] == 'NNS')) or
                    ((wtp1[1] == 'RB' or wtp1[1] == 'RBR' or wtp1[1] == 'RBS') and (wtp2[1] == 'JJ') and (wtp3[1] != 'NN' or wtp3[1] != 'NNS')) or
                    ((wtp1[1] == 'JJ') and (wtp2[1] == 'JJ') and (wtp3[1] != 'NN' or wtp3[1] != 'NNS')) or
                    ((wtp1[1] == 'NN' or wtp1[1] == 'NNS') and (wtp2[1] == 'JJ') and (wtp3[1] != 'NN' or wtp3[1] != 'NNS')) or
                    ((wtp1[1] == 'RB' or wtp1[1] == 'RBR' or wtp1[1] == 'RBS') and (wtp2[1] == 'VB' or wtp2[1] == 'VBD' or wtp2[1] == 'VBN' or wtp2[1] == 'VBG'))):

                    pair_key = f"{wtp1[0]}-{wtp2[0]}"
                    word_pairs.add(pair_key)
                    word_pair_features_for_review[review_id][pair_key] += 1
    
    print("sent features done in %0.3fs." % (time() - t0))
    
    return (word_pair_features_for_review,word_pairs)
        

def generate_pos_distribution_vector(idx, word_pair_features, word_pairs):
    x = []
    for key in word_pairs:
        x.append(word_pair_features[idx][key])
    return np.array(x)

In [20]:
# pos_tagged_reviews = load_pos_tagged_reviews_by_business(pos_tagged_restaurant_reviews_data_path)
# a,b = review_pos_distribution_vector(pos_tagged_reviews)

# print(len(b))

In [7]:
import numpy as np
#transform_review_text_pos_tags(stemmed_restaurant_reviews_data_path,pos_tagged_restaurant_reviews_data_path)
business_reviews = load_stemmed_user_reviews_by_business(restaurant_reviews_data_path)

business_reviews_greater_than_X = {x:business_reviews[x] for x in business_reviews if len(business_reviews[x]) > 50}
print(len(business_reviews_greater_than_X))
reviews = [business_reviews_greater_than_X[x] for x in business_reviews_greater_than_X]
from functools import reduce
all_reviews = reduce(lambda x,y : x + y,reviews,[])
print (len(all_reviews))
# print(all_reviews[:2])
for key in business_reviews_greater_than_X:
    print(key)
    break





1412
127478
u73j2VQ3TGWdMO-AG7MABw


In [21]:
#business_reviews = load_stemmed_user_reviews_by_business(stemmed_restaurant_reviews_data_path)
# reviews_for_one_business = business_reviews['faPVqws-x-5k2CQKDNtHxw']
reviews_for_one_business = all_reviews
doc_vectorizer,lda = extract_topics_using_lda(all_reviews[:100],CountVectorizer,10,1,10)

done in 0.043s.
done in 0.329s.

Topics in LDA model:
Topic 0:
food time good 20 just came chicken people minutes restaurant seated service upgrades said waits don sandwich mins place experience second mac slow list um told half wednesday staff pick palmer ordered waffles bad pleasure 30 glass lo order having wine way busy manager server walking sides hard directly chase
Documents -----
Document 71:
Dealing with the staff is NOT worth the food. 

This used to be my favorite place a few years ago, but after the long waits when dining in I decided to only get carry out. Love their red velvet cake and ordered one 24 hours in advance for my birthday. We came it pick it up 20 mins after they were supposed to have it ready, to find out the bakers never got the order. Was told it would be ready in 5 mins by one staff member, so we waited 10. Only then to be told by the actual baker that it would take at least an hour. He was apologetic and said since we couldn't pick up the cake until after m

In [13]:
tf_vectorizer,nmf = extract_topics_using_nfm(all_reviews,TfidfVectorizer,50,1)

done in 19.713s.
done in 310.970s.

Topics in NFM model:
50
Topic #0: table came asked said ordered waitress manager told didn waiter
Topic #1: great atmosphere service spot fun selection prices food experience loved
Topic #2: sauce ordered dish flavor sweet bread little cream beef fresh
Topic #3: burger fries burgers bun shake onion bacon truffle patty potato
Topic #4: pizza crust pizzas slice pepperoni italian toppings sausage oven bianco
Topic #5: sushi rolls roll ayce fish sashimi fresh tuna nigiri quality
Topic #6: crab shrimp lobster legs seafood oysters king cakes prime rib
Topic #7: buffet buffets selection station price variety desserts line quality prime
Topic #8: place try recommend looking friends eat packed cool fun town
Topic #9: food quality price prices portions better atmosphere authentic indian average
Topic #10: breakfast eggs pancakes brunch toast bacon french benedict hash morning
Topic #11: wait worth long line waiting table seated hours minute busy
Topic #12: tac

In [30]:
restaurant_stars = get_restaurant_stars(restaurant_data_path)
def get_review_topic_vector_for_business(business_id,business_reviews,model,vectorizer):
    reviews_for_business = business_reviews[business_id]
    doc_vector = vectorizer.transform(reviews_for_business)
    review_topic_vectors = model.transform(doc_vector)
    return np.mean(review_topic_vectors,axis=0)

def get_X_and_Y(restaurant_stars,business_reviews,model,vectorizer):
    X = []
    Y = []
    t0 = time()
    i = 0
    for key in business_reviews:
        if key in restaurant_stars:
            i+=1
            if i % 100 == 0:
                
                print(i)
            vector = get_review_topic_vector_for_business(key,business_reviews,model,vectorizer)
            X.append(vector)
            Y.append(restaurant_stars[key])
    
    print("done in %0.3fs." % (time() - t0))
    return np.array(X),np.array(Y)
X,Y = get_X_and_Y(restaurant_stars,business_reviews,lda,doc_vectorizer)
print(X.shape,Y.shape)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [72]:
# business_reviews = load_stemmed_user_reviews_by_business(stemmed_restaurant_reviews_data_path)
# # reviews_for_one_business = business_reviews['faPVqws-x-5k2CQKDNtHxw']
# reviews_for_one_business = all_reviews
# doc_vectorizer,nfm = extract_topics_using_nfm(reviews_for_one_business,TfidfVectorizer,10,100)
# review_doc_vector = doc_vectorizer.transform(reviews_for_one_business)
# review_topic_vector = nfm.transform(review_doc_vector)
# #print(review_topic_vector)

In [31]:
X_Train = X[:40000]
Y_Train = Y[:40000]

X_Test = X[40000:]
Y_Test = Y[40000:]

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from math import sqrt

from sklearn.svm import SVR

def run_supervised(X_Train,Y_Train,X_Test,Y_Test):

    model = SVR(gamma='scale', C=1.0, epsilon=0.2)
    model.fit(X_Train,Y_Train)
    Y_Pred = model.predict(X_Test)
    MSE = mean_squared_error(Y_Test,Y_Pred)
    print(MSE)
    Pearsons_coefficient,p_value = pearsonr(Y_Test,Y_Pred)
    print("MSE " + str(MSE),"Pearsons " + str(Pearsons_coefficient))
    print("RMSE"+ str(sqrt(MSE)))


run_supervised(X_Train,Y_Train,X_Test,Y_Test)

0.601923860746
MSE 0.601923860746 Pearsons 0.490916918015
RMSE0.7758375221309036


In [32]:
np.savetxt("X2.csv", X, delimiter=",")
np.savetxt("Y2.csv",Y,delimiter=',')

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

def extract_topics_using_lda(list_of_text,vectorizer,no_of_topics):
    pass